In [4]:
library(ks)
DATACAPTIONVEC <- c("ID","SEASON","DATE","TEAM1","TEAM2","WON","SCORE","SHOTS","FACEOFF","TAKEAWAY","GIVEAWAY","PIM","HITS","PPG","ATTENDANCE")

Loading required package: KernSmooth
KernSmooth 2.23 loaded
Copyright M. P. Wand 1997-2009
Loading required package: misc3d
Loading required package: mvtnorm
Loading required package: rgl


In [5]:
nhlDataSum=data.frame()
nhlDataDelta=data.frame()
SeasonVector=c(2010)#,2011,2012,2014,2015,2016)
NumberOfSeasons=length(SeasonVector)
for(season in SeasonVector)
{
  tableName=paste("../dataSetsNHL/dataFileNhl_",season,"_regular_sum.dat",sep="")
  nhlDataS=read.table(tableName)
  colnames(nhlDataS) <- DATACAPTIONVEC
  nhlDataSum<-rbind(nhlDataSum,nhlDataS)
  tableName=paste("../dataSetsNHL/dataFileNhl_",season,"_regular_delta.dat",sep="")
  nhlDataS=read.table(tableName)
  colnames(nhlDataS) <- DATACAPTIONVEC
  nhlDataDelta<-rbind(nhlDataDelta,nhlDataS)
}
colnames(nhlDataSum) <- DATACAPTIONVEC
colnames(nhlDataDelta) <- DATACAPTIONVEC

In [6]:
teams <- nhlDataDelta$TEAM1
LISTOFTEAMS=unique(teams)
print("we have the following teams")
print(LISTOFTEAMS)

[1] "we have the following teams"
 [1] BOS       TOR       COL       CGY       CAR       PIT       CHI      
 [8] DET       BUF       NJ        NYI       NYR       WSH       ATLANTA  
[15] FLA       CBJ       DAL       STL       EDM       ANA       LA       
[22] VAN       PHI       MIN       TB        NSH       SJ        PHO      
[29] CANADIENS OTT      
30 Levels: ANA ATLANTA BOS BUF CANADIENS CAR CBJ CGY CHI COL DAL DET ... WSH


In [7]:
nhlDataMatrix=as.matrix(cbind(nhlDataDelta$WON,nhlDataDelta$SHOTS))#,nhlDataDelta$FACEOFF))#,nhlDataSum$SHOTS,nhlDataSum$FACEOFF))

In [8]:
estimateProbability <- function(inputVector)
    {
    Pks=kde(nhlDataMatrix,eval.points=inputVector)
    return(Pks$estimate)
    }
#wins,shotsdifference,faceoffdifference,shotssum,faceoffsum
iVec=c(1,5)#,5)#,50,40)
print(estimateProbability(iVec))

[1] 0.22324


In [9]:
getCondProbability <- function(getWin,PMINV,PMAXV)
    {
    #function calculating the conditional probability of win or loss given a parameter value for which to calculate the 
    # probability exceeds the input value
    #input:
    # getWin - 1 if want to now the probability of winning, else 0 (for loosing)
    # parameterSlotIndex - index for the parameter for which to calculate the PDF
    # parameterValue - value of the parameter which is assumed to be exceeded
    P=0
    p1=getWin
    p2min=PMINV[2];p3min=PMINV[2];p4min=PMINV[3];p5min=PMINV[4];
    p2max=PMAXV[2];p3max=PMAXV[2];p4max=PMAXV[3];p5max=PMAXV[4];
    #always integer steps in our statistics
        print(P)
        for(p2 in p2min:p2max){
          #  for(p3 in p3min:p3max){
               # for(p4 in p4min:p4max){
                   # for(p5 in p5min:p5max)
                   #     {
                            iVec=c(p1,p2)#,p3,p4,p5)
                             #result=kde(nhlDataMatrix,eval.points=iVec)
                             #P=P+result$estimate
                            P=P+estimateProbability(iVec)
                            print(P)
                  #  }
              #  }
           # }
        }
    print(P)
    message=paste("We have found a probability of",P," %")
    print(message)
    #return(message)
    }

In [10]:
#the third column goes from 10 shots to max value of 30 shots
PMINV=c(0,-15,10,30,30)
PMAXV=c(1,15,30,50,50)
getCondProbability(1,PMINV,PMAXV)

[1] 0
[1] 0.05386987
[1] 0.1186611
[1] 0.1974263
[1] 0.2926446
[1] 0.4054476
[1] 0.5352806
[1] 0.6800237
[1] 0.8365848
[1] 1.001971
[1] 1.174484
[1] 1.354304
[1] 1.543027
[1] 1.74248
[1] 1.953546
[1] 2.175523
[1] 2.406115
[1] 2.641935
[1] 2.879234
[1] 3.114531
[1] 3.344922
[1] 3.568163
[1] 3.782657
[1] 3.98738
[1] 4.181667
[1] 4.364881
[1] 4.536141
[1] 4.694265
[1] 4.837949
[1] 4.966011
[1] 5.077635
[1] 5.172696
[1] 5.172696
[1] "We have found a probability of 5.17269625108125  %"
